In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import healpy as hp
from set_dataloader import CCD
import time


#### Experimenting with input sizes

In [ ]:
time_start = time.time()

# For DECAM, BASS, MzLS
with open('../../bricks_data/pixel2ccd_2048_non_inclusive.pickle', 'rb') as f:
    subpixel2ccd_dict = pickle.load(f)
    f.close()

time_end = time.time()
time_passed = time_end - time_start
print()
print(f"{time_passed / 60:.5} minutes ({time_passed:.3} seconds) taken to import the dict")
print()

In [ ]:
print(len(subpixel2ccd_dict))

In [ ]:
time_start = time.time()

# For DECAM, BASS, MzLS
with open('../../bricks_data/pixel2subpixel_256_2048.pickle', 'rb') as f:
    pixel2subpixel_dict = pickle.load(f)
    f.close()

time_end = time.time()
time_passed = time_end - time_start
print()
print(f"{time_passed / 60:.5} minutes ({time_passed:.3} seconds) taken to import the dict")
print()

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max_sub = 0
lens = np.zeros(len(subpixel2ccd_dict))

for i, pix in enumerate(subpixel2ccd_dict.keys()):
    no_ccds = len(subpixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max_sub = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per subpixel:", lens.mean())
print("Max CCD's per subpixel:", max)
print("Min CCD's per subpixel:", min)



In [ ]:
plt.hist(lens, bins=[0, 30, 50, 100])

In [ ]:
with open('../../bricks_data/pixel2ccd_256_non_inclusive.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max = 0
lens = np.zeros(len(pixel2ccd_dict))

for i, pix in enumerate(pixel2ccd_dict.keys()):
    no_ccds = len(pixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per pixel:", lens.mean())
print("Max CCD's per pixel:", max)
print("Min CCD's per pixel:", min)


In [ ]:
plt.hist(lens, bins=[0, 30, 50, 100])

In [ ]:
ccd = CCD()

ra0, dec0, ra1, dec1, ra2, dec2, ra3, dec3 = ccd.get_boundaries()



### Plotting the Pixel - CCD Relationship for the most populated pixel

In [ ]:
#Validating everything went as intended visually

NSIDE = 256
pixel_number = index_max
print(len(pixel2ccd_dict.keys()))

ccd = pixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd:
    xs = [ra0[cc], ra1[cc], ra2[cc], ra3[cc], ra0[cc]]
    ys = [dec0[cc], dec1[cc], dec2[cc], dec3[cc], dec0[cc]]
    plt.plot(xs, ys)

pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(), lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0], pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1], pixel_boundary_raDec[1][0])

plt.plot(pixel_boundary_raDec[0], pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

In [ ]:
#Validating everything went as intended visually

NSIDE = 2048
pixel_number = index_max_sub
print(len(subpixel2ccd_dict.keys()))

ccd_sub = subpixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd_sub:
    #coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
    xs = [ra0[cc], ra1[cc], ra2[cc], ra3[cc], ra0[cc]]
    ys = [dec0[cc], dec1[cc], dec2[cc], dec3[cc], dec0[cc]]
    plt.plot(xs, ys)


pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(), lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0], pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1], pixel_boundary_raDec[1][0])

plt.plot(pixel_boundary_raDec[0], pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

